# Pricing Defaultable Bonds and CDS
This notebook provides an interactive approach to pricing Defaultable Bonds and Credit Default Swaps (CDS) using the Hazard rate model.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

## Hazard Rate Model
The hazard rate, also known as the default intensity, is the instantaneous rate of default at any given time. It is denoted by $ \lambda(t) $.

In [3]:
# Define the hazard rate function
def hazard_rate(t, lambda_0, lambda_1):
    return lambda_0 + lambda_1 * t

However, here we will calibrate the hazard rate based on the market data of CDS prices

## Survival Probability
The survival probability is the probability that the entity has not defaulted by time $ t $. It is given by:
$$
S(t) = e^{-\int_0^t \lambda(u) du}
$$

In discrete time frame:
$$
S(t) = (1-h(t))*S(t-1)
$$

In [6]:
# Define the survival probability function
def survival_probability(t, hazard_rate_params):
    if t == 0:
        return 1
    else:
        return (1 - hazard_rate_params[t-1]) * survival_probability(t-1, hazard_rate_params)

## Pricing Defaultable Bonds
The price of a defaultable bond can be calculated by discounting the expected cash flows, taking into account the survival probability.

In [ ]:
# Define the bond pricing function
def price_defaultable_bond(face_value, coupon_rate, maturity, compounding_periods_per_annum, discount_rate, recovery_rate, hazard_rates):
    price = 0
    recovery = recovery_rate * face_value
    hazard_rate_params = np.repeat(hazard_rates / compounding_periods_per_annum, compounding_periods_per_annum)
    for t in range(1, maturity * compounding_periods_per_annum + 1):
        coupon = face_value * coupon_rate
        survival_prob = survival_probability(t, hazard_rate_params)
        default_prob = hazard_rate_params[t-1] * survival_probability(t - 1, hazard_rate_params)
        price += (recovery * default_prob + coupon * survival_prob) / (1 + discount_rate / compounding_periods_per_annum) ** t
    # Add the face value discounted by the survival probability at maturity
    price += (face_value * survival_prob) / (1 + discount_rate / compounding_periods_per_annum) ** (maturity * compounding_periods_per_annum)
    return price

Example of Defaultable bond pricing

In [15]:
# Assuming a constant Hazard rate

face_value = 100
coupon_rate = 0.05
maturity = 1
compounding_periods_per_annum = 2
hazard_rates_annual = np.array([0.04] * maturity)
discount_rate = 0.05
recovery_rate = 0.1
price = price_defaultable_bond(face_value, coupon_rate, maturity, compounding_periods_per_annum, discount_rate, recovery_rate, hazard_rates_annual)
print(f"The price of the defaultable bond is: {price}") 

The price of the defaultable bond is: 101.14503271861987


## Pricing Credit Default Swaps (CDS)
A CDS is a financial derivative that provides protection against the default of a reference entity. The pricing involves calculating the premium leg and the protection leg.

In [22]:
# Define the CDS pricing function
def price_cds(notional, spread_basis, maturity, compounding_periods_per_annum, discount_rate, recovery_rate, hazard_rates):
    premium_leg = 0
    protection_leg = 0
    hazard_rate_params = np.repeat(hazard_rates / compounding_periods_per_annum, compounding_periods_per_annum)
    for t in range(1, maturity * compounding_periods_per_annum + 1):
        survival_prob = survival_probability(t, hazard_rate_params)
        default_prob = hazard_rate_params[t-1] * survival_probability(t - 1, hazard_rate_params)
        premium_leg += 0.0001 * (spread_basis/compounding_periods_per_annum) * notional * survival_prob / (1 + discount_rate/compounding_periods_per_annum) ** t
        interest_compounded = 0.0001 * (1/2) * notional * (spread_basis/compounding_periods_per_annum) / (1 + discount_rate/compounding_periods_per_annum) ** t   # Assuming the default happens in the middle of a period
        premium_leg += interest_compounded * default_prob
        protection_leg += notional*(1-recovery_rate) * default_prob / (1 + discount_rate/compounding_periods_per_annum) ** t
    return protection_leg - premium_leg

## Example Usage
Let's calculate the price of a CDS with the following parameters:

In [28]:
# Parameters


maturity = 2
discount_rate = 0.01
notional = 1000000
spread = 218.88953946553
periods = 4
recovery_rate = 0.45
hazard_rates_annual = np.array([0.040281146080,0.038886206677])

price = price_cds(notional, spread, maturity, periods, discount_rate, recovery_rate, hazard_rates_annual)
print(f"The price of the CDS is: {price}")

The price of the CDS is: -3.821551217697561e-07


In [32]:
# Parameters


maturity = 5
discount_rate = 0.05
notional = 10000000
spread = 301.507537688442
periods = 4
recovery_rate = 0.25
hazard_rates_annual = np.array([0.04] * maturity)

price = price_cds(notional, spread, maturity, periods, discount_rate, recovery_rate, hazard_rates_annual)
print(f"The price of the CDS is: {price}")

The price of the CDS is: 4.656612873077393e-10


In [34]:
from scipy.optimize import minimize

# Objective function for hazard rate optimization
def hazard_rate_objective(hazard_rates_annual, face_value, coupon_rate, maturity, compounding_periods_per_annum, discount_rate, recovery_rate, target_price):
    price = price_defaultable_bond(face_value, coupon_rate, maturity, compounding_periods_per_annum, discount_rate, recovery_rate, hazard_rates_annual)
    return (price - target_price)**2  # We want the price to be as close to the target price as possible

# Target price for the defaultable bond
target_price = 100

# parameters
face_value = 100
coupon_rate = 0.05
maturity = 1
compounding_periods_per_annum = 2
discount_rate = 0.05
recovery_rate = 0.1

# Initial guess for the hazard rates
initial_hazard_rates = np.array([0.04] * maturity)

# Perform the optimization for hazard rates
result_hazard_rates = minimize(hazard_rate_objective, initial_hazard_rates, args=(face_value, coupon_rate, maturity, compounding_periods_per_annum, discount_rate, recovery_rate, target_price))
optimal_hazard_rates = result_hazard_rates.x
print(f"Optimal hazard rates: {optimal_hazard_rates}")

# Objective function for CDS spread optimization
def cds_spread_objective(spread, notional, maturity, periods, discount_rate, recovery_rate, hazard_rates_annual):
    price = price_cds(notional, spread, maturity, periods, discount_rate, recovery_rate, hazard_rates_annual)
    return abs(price)  # We want the price to be as close to 0 as possible

# Parameters

maturity = 5
discount_rate = 0.05
notional = 10000000
periods = 4
recovery_rate = 0.25
hazard_rates_annual = np.array([0.04] * maturity)


# Initial guess for the spread
initial_spread = 200

# Perform the optimization for CDS spread
result_spread = minimize(cds_spread_objective, initial_spread, args=(notional, maturity, periods, discount_rate, recovery_rate, hazard_rates_annual))
optimal_spread = result_spread.x[0]
print(f"Optimal CDS spread: {optimal_spread}")



Optimal hazard rates: [0.05263157]
Optimal CDS spread: 301.50753768518763
